In [29]:
import torch
import torch.nn as nn
import numpy as np

In [ ]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [74, 66, 43],
                   [91, 87, 65],
                   [88, 134, 59],
                   [101, 44, 37],
                   [68, 96, 71],
                   [73, 66, 44],
                   [92, 87, 64],
                   [87, 135, 57],
                   [103, 43, 36],
                   [68, 97, 70]],dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119],
                    [57, 69],
                    [80, 102],
                    [118, 132],
                    [21, 38],
                    [104, 118],
                    [57, 69],
                    [82, 100],
                    [118, 134],
                    [20, 38],
                    [102, 120]],dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [16]:
targets, inputs

(tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.],
         [ 57.,  69.],
         [ 80., 102.],
         [118., 132.],
         [ 21.,  38.],
         [104., 118.],
         [ 57.,  69.],
         [ 82., 100.],
         [118., 134.],
         [ 20.,  38.],
         [102., 120.]]),
 tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.],
         [ 74.,  66.,  43.],
         [ 91.,  87.,  65.],
         [ 88., 134.,  59.],
         [101.,  44.,  37.],
         [ 68.,  96.,  71.],
         [ 73.,  66.,  44.],
         [ 92.,  87.,  64.],
         [ 87., 135.,  57.],
         [103.,  43.,  36.],
         [ 68.,  97.,  70.]]))

In [18]:
from torch.utils.data import TensorDataset

In [22]:
from torch.utils.data import DataLoader

In [ ]:
#define Dataset
# TensorDataset allows us to use inputs as tuples
# we can't change/modify the elements in tuple
train_ds = TensorDataset(inputs, targets)
train_ds[0:3] # this lines gives us the first 3 rows of inputs and targets

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [23]:
#define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [24]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 74.,  66.,  43.],
        [ 69.,  96.,  70.],
        [101.,  44.,  37.],
        [ 68.,  97.,  70.],
        [ 73.,  66.,  44.]])
tensor([[ 57.,  69.],
        [103., 119.],
        [ 21.,  38.],
        [102., 120.],
        [ 57.,  69.]])


In [25]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [31]:
#define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.5205,  0.5674,  0.4259],
        [-0.0420,  0.1170, -0.1215]], requires_grad=True)
Parameter containing:
tensor([ 0.1262, -0.4069], requires_grad=True)


In [32]:
#parameter
list(model.parameters())

[Parameter containing:
 tensor([[-0.5205,  0.5674,  0.4259],
         [-0.0420,  0.1170, -0.1215]], requires_grad=True),
 Parameter containing:
 tensor([ 0.1262, -0.4069], requires_grad=True)]

In [33]:
#generate predictions
preds = model(inputs)
preds

tensor([[ 18.4598,  -0.8591],
        [ 29.9503,  -1.7101],
        [ 55.5774,   4.5706],
        [-12.8076,  -4.1568],
        [ 48.4958,  -0.5789],
        [ 17.3719,  -1.0181],
        [ 29.8088,  -1.9486],
        [ 55.4828,   4.4070],
        [-11.7197,  -3.9978],
        [ 49.4422,  -0.6584],
        [ 18.3183,  -1.0977],
        [ 28.8624,  -1.8691],
        [ 55.7189,   4.8091],
        [-13.7540,  -4.0773],
        [ 49.5837,  -0.4198]], grad_fn=<AddmmBackward0>)

In [35]:
#import nn.functional
import torch.nn.functional as F

In [36]:
#define loss function
loss_fn = F.mse_loss

In [38]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(6016.4111, grad_fn=<MseLossBackward0>)


In [40]:
#define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [50]:
#utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):

    #repeats for given numbers of epochs
    for epoch in range(num_epochs):

        #train with batches of data
        for xb,yb in train_dl:  

            #1. generate prediction
            pred = model(xb)

            #2. Calculate loss
            loss = loss_fn(pred, yb)

            #3. compute gradients
            loss.backward()

            #4. update parameters using gardients
            opt.step()

            #5. reset the gradients to zero
            opt.zero_grad()

        #print the progress
        if(epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [51]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 4.2097
Epoch [20/100], Loss: 4.2491
Epoch [30/100], Loss: 2.9147
Epoch [40/100], Loss: 1.6601
Epoch [50/100], Loss: 4.2203
Epoch [60/100], Loss: 2.9274
Epoch [70/100], Loss: 2.6606
Epoch [80/100], Loss: 2.5466
Epoch [90/100], Loss: 2.4614
Epoch [100/100], Loss: 2.5447


In [53]:
#generating predictions
preds = model(inputs)
preds

tensor([[ 56.9879,  70.5237],
        [ 81.8370,  99.4717],
        [118.5490, 135.3947],
        [ 20.9096,  38.4745],
        [101.4945, 116.1177],
        [ 55.7366,  69.4229],
        [ 81.6604,  99.3501],
        [118.8239, 135.8828],
        [ 22.1609,  39.5753],
        [102.5692, 117.0969],
        [ 56.8113,  70.4021],
        [ 80.5857,  98.3709],
        [118.7256, 135.5163],
        [ 19.8349,  37.4953],
        [102.7458, 117.2185]], grad_fn=<AddmmBackward0>)

In [54]:
#comparing with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [55]:
#generating pprediction for 6th region
model(torch.tensor([[75, 63, 44.]]))

tensor([[53.4575, 67.3452]], grad_fn=<AddmmBackward0>)

In [1]:
model2 = nn.Sequential(
    nn.Linear(3, 4),
    nn.Sigmoid(),
    nn.Linear(4, 2)
)

NameError: name 'nn' is not defined

In [58]:
opt = torch.optim.SGD(model2.parameters(), lr=1e-4)

In [59]:
fit(100, model2, F.mse_loss, opt, train_dl)

Epoch [10/100], Loss: 7659.7930
Epoch [20/100], Loss: 9448.5439
Epoch [30/100], Loss: 6602.6494
Epoch [40/100], Loss: 5677.1943
Epoch [50/100], Loss: 7344.3311
Epoch [60/100], Loss: 5645.2920
Epoch [70/100], Loss: 9943.1953
Epoch [80/100], Loss: 7621.4038
Epoch [90/100], Loss: 8276.8418
Epoch [100/100], Loss: 9460.2402
